# EE 5020 Homework 2: Statistical inference (frequentist)

Name: FARZANA YASMIN BOBY

CIN: 401413256

## Overview

In this homework, you will practice applying your statistical inference skills on different datasets within the book. Like the previous homework's Section 2, this set of problems will be presented as datasets and associated research questions for you to answer with your own Markdown and Code cells.

## Rubric for this homework

Make sure you write down the reasoning and method of creation for any additional columns or transformations of data you need to create.  Additionally, make sure to justify any conclusions you make to answer questions with statistical tests.  Make sure you justify the statistical test chosen as well (for instance, unpaired vs. paired t-test).

- 25% organization and flow of justification for statistical inference
- 25% organization and flow of Python code
- 25% correct statistical inference
- 25% correct Python code

## Global imports

Write your imports here so you don't have to write imports below.

In [1]:
import numpy as np
import scipy.stats
import pandas as pd
from typing import Tuple
import matplotlib.pyplot as plt

## Problem 1

**Dataset:** `as_datasets/hof.csv`

**Dataset description:** A data set containing number of hits and hall of fame status of various baseball players

**Write and discuss the steps to answering the following research question:** Subsample the full dataset again by the last two digits of your CIN floor divided by 4. Is it more likely for a hall of famer to have more hits than a non-hall of famer?

Subsampling the dataset df_hof by the last two digit of my CIN and renaming it to df_subsampled

In [2]:
df_hof = pd.read_csv("../as_datasets/hof.csv")

def cin_subsample_dataframe(df_input: pd.DataFrame, cin: int) -> pd.DataFrame:
    """
    Returns subsampled dataframe based on last two digits of cin divided by 4.
    """
    selection_vector = np.arange(0, len(df_input), (cin % 100) // 4)
    return df_input.iloc[selection_vector].copy()

df_subsampled = cin_subsample_dataframe(df_hof, 401413256)

Now, producing two new DataFrames with hall of famers and non hall of famers and renaming them df_hall_of_famer and df_non_hall_of_famer

In [3]:
df_hall_of_famer = df_subsampled[df_subsampled['HOF'] == 1]
print(df_hall_of_famer.head())

df_non_hall_of_famer = df_subsampled[df_subsampled['HOF'] == 0]
print(df_non_hall_of_famer.head())

     Hits  HOF
434  1659    1
518  1840    1
574  2004    1
672  2313    1
686  2383    1
    Hits  HOF
0    972    0
14  1068    0
28  1105    0
42  1132    0
56  1154    0


Creating a function for getting confidence interval

In [4]:
def get_confidence_interval(dataset: pd.Series, ci_level: float = 0.95, force_t: bool = False) -> Tuple[float, float]:
    """
    Returns the confidence interval for the given data series, based on the 
      z-distribution if the number of samples > 30 and the t-distribution if
      the number of samples is less than or equal to 30.

    :param dataset: a single series of data to get the confidence interval for the mean.
    :param ci_level: level for the confidence interval
    :param force_t: True if forced to use t distribution
    """
    n = len(dataset)
    mean = dataset.mean()
    stdev = dataset.std()
    stderr = stdev / np.sqrt(n)
    if n > 30 and not force_t:
        return scipy.stats.norm.interval(ci_level, mean, stderr)
    else:
        ddof = n - 1
        return scipy.stats.t.interval(ci_level, ddof, mean, stderr)

Now, getting confidence interval of number of hits for both hall of famers and non hall of famers with the help of the function and also performin individual t-test

In [5]:
print(f"The means for famers are: {df_hall_of_famer.mean()}")
print(f"the means for non famers are: {df_non_hall_of_famer.mean()}")

print(f"The 95% confidence interval of no. of hits for hall of famers is: {get_confidence_interval(force_t=True, dataset=df_hall_of_famer['Hits'])}")
print(f"The 95% confidence interval of no. of hits for non hall of famers is: {get_confidence_interval(force_t=True, dataset=df_non_hall_of_famer['Hits'])}")
scipy.stats.ttest_ind(df_hall_of_famer['Hits'], df_non_hall_of_famer['Hits'])

The means for famers are: Hits    2504.444444
HOF        1.000000
dtype: float64
the means for non famers are: Hits    1572.425532
HOF        0.000000
dtype: float64
The 95% confidence interval of no. of hits for hall of famers is: (2012.2853905799304, 2996.6034983089585)
The 95% confidence interval of no. of hits for non hall of famers is: (1457.4585510849813, 1687.3925127448058)


Ttest_indResult(statistic=5.85595205321475, pvalue=2.899670803354434e-07)

It shows that the mean number of hits for a player who is in hall of fame is 2504.444444, and for a player of non hall of fame is 1572.425532. So, the mean hits for hall of famers is higher than non hall of famers.It's also seen that 95% of the total population of no. of hits for hall of famers lies within the range 2012.285 to 2996.603, and for non hall of famers the value lies within 1457.458 to 1687.392. After performing t-test, the result shows p value is quite smaller (<0.05). so, there is mean difference between hall of famer and non hall of famer. It's more likely for a hall of famer to have more hits than a non-hall of famer.

## Problem 2

**Dataset:** `as_datasets/bac.csv`

**Dataset description:** Data on blood alcohol content (BAC) and whether the subject passed or failed the standardized field sobriety test

**Write and discuss the steps to answering the following research question:** How accurate is a police officer's sobriety test? In other words, do subjects who fail the sobriety test have higher BAC than subjects who passed the sobriety test?

In [6]:
df_bac = pd.read_csv("../as_datasets/bac.csv")
df_bac.head()

,Subject,BAC,PASS,forpass,pred,pass.ols,PASS.OLS
0,54,0.000344,1,0.518128,0.971148,0,0
1,94,0.000523,1,0.464620,0.970857,0,0
2,72,0.001530,1,0.797098,0.969166,0,1
3,1,0.001819,1,0.327069,0.968663,0,0
4,49,0.001850,1,0.546285,0.968608,0,0


Now, producing two new DataFrames with subjects who fails sobriety test and subjects who passes sobriety test and renaming them df_fails and df_pass

In [7]:
df_fails = df_bac[df_bac['PASS'] == 0]
print(df_fails.head())

df_pass = df_bac[df_bac['PASS'] == 1]
print(df_pass.head())

    Subject       BAC  PASS   forpass      pred  pass.ols  PASS.OLS
9        40  0.004256     0  0.944598  0.964099         0         0
13       41  0.010262     0  0.903742  0.949961         0         0
17       13  0.013762     0  0.927770  0.939433         0         0
34       20  0.036139     0  0.843834  0.809930         0         0
38       80  0.038245     0  0.903805  0.790499         0         0
   Subject       BAC  PASS   forpass      pred  pass.ols  PASS.OLS
0       54  0.000344     1  0.518128  0.971148         0         0
1       94  0.000523     1  0.464620  0.970857         0         0
2       72  0.001530     1  0.797098  0.969166         0         1
3        1  0.001819     1  0.327069  0.968663         0         0
4       49  0.001850     1  0.546285  0.968608         0         0


Getting confidence interval using the function and performing t-test for subjects who fails sobriety test and who passes sobriety test

In [23]:
print(f"The means for subjects who fails sobriety test are: {df_fails.mean()}")
print(f"The means for subjects who passes sobriety test are: {df_pass.mean()}")

print(f"The 95% confidence interval of BAC for subjects who fails sobriety test is: {get_confidence_interval(force_t=True, dataset=df_fails['BAC'])}")
print(f"The 95% confidence interval of BAC for subjects who passess sobriety test is: {get_confidence_interval(force_t=True, dataset=df_pass['BAC'])}")
scipy.stats.ttest_ind(df_pass['BAC'], df_fails['BAC'])

The means for subjects who fails sobriety test are: Subject     50.076923
BAC          0.070161
PASS         0.000000
forpass      0.674407
pred         0.384627
pass.ols     0.000000
PASS.OLS     0.000000
dtype: float64
The means for subjects who passes sobriety test are: Subject     50.770492
BAC          0.034829
PASS         1.000000
forpass      0.429993
pred         0.754091
pass.ols     0.000000
PASS.OLS     0.245902
dtype: float64
The 95% confidence interval of BAC for subjects who fails sobriety test is: (0.062070381025131974, 0.07825143394922703)
The 95% confidence interval of BAC for subjects who passess sobriety test is: (0.028862896703866825, 0.04079505936170695)


Ttest_indResult(statistic=-7.194405281581035, pvalue=1.2700067956852572e-10)

The results show that, the mean BAC for subjects who fails the sobriety test is 0.070161 and for subjects who passes the test is 0.034829. The BAC for sobriety test failure is higher. 95% of the data for BAC lies within 0.0620... to 0.0782.. for sobriety test failure and within 0.0288... to 0.0407... for sobriety test passing. the range for BAC for sobriety test failure is higher. After performing t-test it shows a much lower pvalue (<0.05), therefore, there is a BAC difference. So, we can say that subjects who fail the sobriety test have higher BAC than subjects who passed the sobriety test.

## Problem 3

**Dataset:** `as_datasets/caffeine.csv`

**Dataset description:** Endurance times of 9 athletes when given 5 mg and 13 mg of caffeine

**Write and discuss the steps to answering the following research question:** Does a higher concentration of actually prolong the endurance time of an athlete?

In [9]:
df_caffeine = pd.read_csv("../as_datasets/caffeine.csv")
df_caffeine.head()

,mg5,mg13
0,35.05,36.20
1,44.50,46.48
2,73.25,69.47
3,66.20,70.54
4,42.47,37.55


Getting confidence interval for both concentrations and applying rel t test as all the athletes are given both concentrations of caffeine

In [10]:
print(f"The means are: {df_caffeine.mean()}")
print(f"The 95% confidence interval for mg5 is: {get_confidence_interval(force_t=True, dataset=df_caffeine['mg5'])}")
print(f"The 95% confidence interval for mg13 is: {get_confidence_interval(force_t=True, dataset=df_caffeine['mg13'])}")
scipy.stats.ttest_rel(df_caffeine['mg5'], df_caffeine['mg13'])

The means are: mg5     57.676667
mg13    58.148889
dtype: float64
The 95% confidence interval for mg5 is: (45.37600719147748, 69.97732614185587)
The 95% confidence interval for mg13 is: (46.51574593835636, 69.78203183942139)


Ttest_relResult(statistic=-0.12052261484527026, pvalue=0.9070411564076122)

We can see that 95% of population for concentration mg5 lies within 45.376... to 69.977... and for mg13 it's 46.515.. to 69.782... , which is close to one another. The t-test shows the statistics isn't significant and the p-value is higher than 0.05. So,  a higher concentration actually doesn't prolong the endurance time of an athlete.

## Problem 4

**Dataset:** `as_datasets/rent.csv`

**Dataset description:** Monthly rent of 10 randomly selected apartments with less than 100 sq ft in 6 American cities

**Write and discuss the steps to answering the following research question:** Which cities have similar monthly rent means?

In [11]:
df_rent = pd.read_csv("../as_datasets/rent.csv")
df_rent.head()

,Santa Monica CA,Boise ID,Tucson AZ,Detroit MI,Pittsburgh PA,Orlando FL
0,10230,1600,2495,3195,2480,2242
1,10000,1500,2200,2695,2435,2000
2,9000,1029,2150,2595,2405,1912
3,8500,1025,1800,2495,2350,1895
4,8250,980,1650,2495,2320,1800


Let's perform pairwise comparison of the mean of monthly rent of the cities by pairwise_tukey.hsd

In [12]:
print(f"The means are: {df_rent.mean()}")
# First, we need to get a "long-form grouped" version of df_rent:
df_rent["apartment_ID"] = df_rent.index + 1000000  # NOTE: normally dataset would have a unique identifier for each sample of data
df_rent_long = df_rent.melt(id_vars=["apartment_ID"],
                            value_vars=[col_label for col_label in df_rent.columns],
                            var_name="City_name",
                            value_name="rent")
df_rent_long.head()

The means are: Santa Monica CA    7929.5
Boise ID           1078.4
Tucson AZ          1789.5
Detroit MI         2096.5
Pittsburgh PA      2344.1
Orlando FL         1825.9
dtype: float64


,apartment_ID,City_name,rent
0,1000000,Santa Monica CA,10230
1,1000001,Santa Monica CA,10000
2,1000002,Santa Monica CA,9000
3,1000003,Santa Monica CA,8500
4,1000004,Santa Monica CA,8250


In [13]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd

rent_hsd = pairwise_tukeyhsd(endog=df_rent_long['rent'], groups=df_rent_long["City_name"])

rent_hsd.summary()

group1,group2,meandiff,p-adj,lower,upper,reject
Boise ID,Detroit MI,1018.1,0.031,60.3579,1975.8421,True
Boise ID,Orlando FL,747.5,0.2096,-210.2421,1705.2421,False
Boise ID,Pittsburgh PA,1265.7,0.0034,307.9579,2223.4421,True
Boise ID,Santa Monica CA,6851.1,0.001,5893.3579,7808.8421,True
Boise ID,Tucson AZ,711.1,0.2576,-246.6421,1668.8421,False
Detroit MI,Orlando FL,-270.6,0.9,-1228.3421,687.1421,False
Detroit MI,Pittsburgh PA,247.6,0.9,-710.1421,1205.3421,False
Detroit MI,Santa Monica CA,5833.0,0.001,4875.2579,6790.7421,True
Detroit MI,Tucson AZ,-307.0,0.9,-1264.7421,650.7421,False
Orlando FL,Pittsburgh PA,518.2,0.59,-439.5421,1475.9421,False


From the above result it's seen that, Boise ID	Orlando FL, Boise ID	Tucson AZ, Detroit MI	Orlando FL, Detroit MI	Pittsburgh PA, Detroit MI	Tucson AZ, Orlando FL	Pittsburgh PA, Orlando FL	Tucson AZ, Pittsburgh PA	Tucson AZ, these pair of cities have almost similar monthly rent means. There exists small mean difference.

## Problem 5

**Dataset:** `as_datasets/gpa.csv`

**Dataset description:** A toy data set containing the heights and gpa of 35 students

**Write and discuss the steps to answering the following research question:** Is there a GPA difference between with students in each quartile (25%) of height?  In other words, do students who are in the bottom 25% of height have a different GPA than students who are in the 25-50% or 50-75% or 75-100% of height?

In [14]:
df_gpa = pd.read_csv("../as_datasets/gpa.csv")
df_gpa.describe()

,height,gpa
count,35.000000,35.000000
mean,66.814286,2.971714
std,4.340449,0.535040
min,57.000000,1.500000
25%,63.500000,2.725000
50%,67.000000,3.000000
75%,70.000000,3.330000
max,74.000000,3.860000


Getting the quartiles of height and then creating new DataFrames with 0-25%, 25-50%, 50-75%, 75-100% of heights and their corresponding gpa

In [15]:
first_quartile = np.quantile(df_gpa['height'], .25)
second_quartile = np.quantile(df_gpa['height'], .5)
third_quartile = np.quantile(df_gpa['height'], .75)

In [19]:
df_first_quart = df_gpa[df_gpa['height'] > first_quartile]
print(f"first_quart:\n{df_first_quart.head()}")

df_second_quart = df_gpa[(first_quartile < df_gpa['height']) & (df_gpa['height'] <= second_quartile)]
print(f"second_quart:\n{df_second_quart.head()}")

df_third_quart = df_gpa[(second_quartile < df_gpa['height']) & (df_gpa['height'] <= third_quartile)]
print(f"third_quart:\n{df_third_quart.head()}")

df_last_quart = df_gpa[df_gpa['height'] > third_quartile]
print(f"last_quart:\n{df_last_quart.head()}")

first_quart:
   height   gpa
0    66.0  2.90
2    64.5  3.62
4    69.5  3.45
5    65.0  2.80
7    68.0  2.81
second_quart:
    height   gpa
0     66.0  2.90
2     64.5  3.62
5     65.0  2.80
9     64.0  2.75
12    66.0  2.49
third_quart:
    height   gpa
4     69.5  3.45
7     68.0  2.81
10    69.0  3.86
11    70.0  1.50
14    69.0  2.70
last_quart:
    height   gpa
13    73.0  3.10
17    72.0  2.84
18    72.0  2.85
19    73.5  3.33
28    71.5  3.20


Now, getting confidence interval for the gpa of each quartiles and performing oneway ANOVA test to see if they have similar or different means

In [20]:
print(f"The means are: {df_first_quart.mean(), df_second_quart.mean(), df_third_quart.mean(), df_last_quart.mean()}")

print(f"The 95% confidence interval for 1st quartile is: {get_confidence_interval(force_t=True, dataset=df_first_quart['gpa'])}")
print(f"The 95% confidence interval for 2nd quartile is: {get_confidence_interval(force_t=True, dataset=df_second_quart['gpa'])}")
print(f"The 95% confidence interval for 3rd quartile is: {get_confidence_interval(force_t=True, dataset=df_third_quart['gpa'])}")
print(f"The 95% confidence interval for last quartile is: {get_confidence_interval(force_t=True, dataset=df_last_quart['gpa'])}")

scipy.stats.f_oneway(df_first_quart['gpa'], df_second_quart['gpa'], df_third_quart['gpa'], df_last_quart['gpa'])

The means are: (height    68.769231
gpa        2.979231
dtype: float64, height    65.750
gpa        3.111
dtype: float64, height    68.9375
gpa        2.8050
dtype: float64, height    72.37500
gpa        2.98875
dtype: float64)
The 95% confidence interval for 1st quartile is: (2.770526239170903, 3.1879352992906362)
The 95% confidence interval for 2nd quartile is: (2.8031757178342227, 3.418824282165777)
The 95% confidence interval for 3rd quartile is: (2.166504536803136, 3.4434954631968635)
The 95% confidence interval for last quartile is: (2.763514220414494, 3.213985779585506)


F_onewayResult(statistic=0.5161712753320405, pvalue=0.6731251730511894)

It's clear from the test result that the statistics is not significant. The p value is higher (>0.05). So, there is no significant GPA difference between with students in each quartile (25%) of height. Students who are in the bottom 25% of height don't have different GPA than students who are in the 25-50% or 50-75% or 75-100% of height. In other words, GPA doesn't depend on the heights of students.